# Preprocessing steps for _RICH_ models

In [1]:
import os
from IPython.display import Markdown

PARTICLE = os.environ.get("PARTICLE", "muon")
LOCAL = os.environ.get("LOCAL", "yes").lower() in ['y', 'yes', 'true', '1']
SAMPLE = os.environ.get("SAMPLE", "2016MU")

Markdown(f"""
 * PARTICLE: `{PARTICLE}`
 * LOCAL: `{LOCAL}`
 * SAMPLE: `{SAMPLE}`
""")


 * PARTICLE: `muon`
 * LOCAL: `True`
 * SAMPLE: `2016MU`


## Technologies and libraries

On top of the standard Python echosystem we are using:
 * `uproot` to convert data from `ROOT TTrees` to `pandas DataFrames`
 * `dask DataFrame` to enable processing datasets larger than the available RAM. Dask takes care of flushing from disk to RAM the data, converting from ROOT to pandas data format on demand.
 * `Arrow Feather` data format to cache in local storage the training dataset
     * Note that custom wrappers to Dask and TensorFlow, as defined in `feather_io.py` are needed 

In [2]:
import uproot
import numpy as np
import pandas as pd
import dask.dataframe as ddf

## Loading data

In [3]:
from glob import glob

if LOCAL:
    if SAMPLE == "2016MU":
        file_pattern = "/tmp/LamarrTraining-j109*.root"
        default_file_list = glob(file_pattern)[:3]
else:
    # file_pattern = "/workarea/cloud-storage/anderlinil/LamarrBenderTrain/j109/*.root"
    default_file_list = []
    
file_list = os.environ.get("INPUT_FILES", " ".join(default_file_list)).split(" ")

print (f"Found {len(file_list)} data files")  

Found 1 data files


### Conversion from `ROOT TTree` to `Pandas DataFrame`

In [4]:
selections = [
    "probe_Brunel_P > 3000",
    "probe_Brunel_P < 200000", 
    "probe_Brunel_ETA > 1.5",
    "probe_Brunel_ETA < 5.5", 
    "probe_Brunel_RichDLLe  > -150",
    "probe_Brunel_RichDLLmu > -150",
    "probe_Brunel_RichDLLk  > -150",
    "probe_Brunel_RichDLLp  > -150",
    "probe_Brunel_RichDLLe  <  150",
    "probe_Brunel_RichDLLmu <  150",
    "probe_Brunel_RichDLLk  <  150",
    "probe_Brunel_RichDLLp  <  150",
    "nTracks_Brunel > 0",
    "nTracks_Brunel < 1000", 
]

cuts = " and ".join([f"({s})" for s in selections])

In [5]:
from functools import partial

opened_files = [uproot.open(f) for f in file_list]

sim = (
    ddf.from_map(
        lambda f: pd.DataFrame(f[f"PidTupler/pid_{PARTICLE}"].arrays(library='np')),
        opened_files,
    ).query(cuts)
)


IsADirectoryError: [Errno 21] Is a directory: '/home/private/lamarr/lb-pidsim-train/pidgan/workflow/notebooks'

### List of features

In [ ]:
from IPython.display import HTML

HTML ("<UL>" + "\n".join(f"<LI> <PRE>{var}</PRE> " for var in sim.columns) + "</UL>")

## Features

In [ ]:
sim["p_GeV"] = sim.probe_Brunel_P/1e3
# sim["log10_p"] = np.log10(sim.probe_Brunel_P)
sim["eta"] = sim.probe_Brunel_ETA
sim = sim.map_partitions(lambda df: df.assign(nTracks_f = df.nTracks_Brunel + np.random.uniform(-0.5, 0.5, len(df))))
sim["charge"] = sim.probe_Brunel_trackcharge

sim["RichDLLe"] = sim.probe_Brunel_RichDLLe
sim["RichDLLmu"] = sim.probe_Brunel_RichDLLmu
sim["RichDLLk"] = sim.probe_Brunel_RichDLLk
sim["RichDLLp"] = sim.probe_Brunel_RichDLLp

In [ ]:
real_conditions = ["p_GeV", "eta", "nTracks_f"]
flag_conditions = ["charge"]

conditions = real_conditions + flag_conditions
target = ["RichDLLe", "RichDLLmu", "RichDLLk", "RichDLLp"]

In [ ]:
from IPython.display import HTML

def list_vars(title, var_list):
    display(HTML (f"<P><B>{title}</B><UL>" + "\n".join(f"<LI> <PRE>{var}</PRE> " for var in var_list) + "</UL><HR>"))

list_vars("Input features (real)", real_conditions)
list_vars("Input features (boolean)", flag_conditions)
list_vars("Output features (real)", target)

In [ ]:
dfs = dict()
npartitions=min(len(file_list), 10)

dfs = sim[conditions + target].head(500_000, npartitions=npartitions)
print(f"Number of {PARTICLE}s:", len(dfs))

### Histograms

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(24, 5), dpi=100)

plt.subplot(1, 3, 1)
plt.xlabel(r"$p$ [GeV/$c$]", fontsize=12)
plt.ylabel(r"Candidates", fontsize=12)
plt.hist(dfs["p_GeV"], bins=np.linspace(1.0, 101.0, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)

plt.subplot(1, 3, 2)
plt.xlabel(r"$\eta$", fontsize=12)
plt.ylabel("Candidates", fontsize=12)
plt.hist(dfs["eta"], bins=np.linspace(1.5, 5.5, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)

plt.subplot(1, 3, 3)
plt.xlabel(r"$\mathtt{nTracks}$", fontsize=12)
plt.ylabel("Candidates", fontsize=12)
plt.hist(dfs["nTracks_f"], bins=np.linspace(0.0, 500.0, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)

plt.show()

In [ ]:
plt.figure(figsize=(24, 10), dpi=100)

plt.subplot(2, 3, 1)
plt.xlabel(r"$\mathtt{RichDLLe}$", fontsize=12)
plt.ylabel(r"Candidates", fontsize=12)
plt.hist(dfs["RichDLLe"], bins=np.linspace(-150.0, 100.0, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)
plt.yscale("log")

plt.subplot(2, 3, 2)
plt.xlabel(r"$\mathtt{RichDLLmu}$", fontsize=12)
plt.ylabel(r"Candidates", fontsize=12)
plt.hist(dfs["RichDLLmu"], bins=np.linspace(-100.0, 150.0, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)
plt.yscale("log")

plt.subplot(2, 3, 3)
plt.xlabel(r"$\mathtt{RichDLLmue}$", fontsize=12)
plt.ylabel(r"Candidates", fontsize=12)
plt.hist(dfs["RichDLLmu"] - dfs["RichDLLe"], bins=np.linspace(-50.0, 150.0, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)
plt.yscale("log")

plt.subplot(2, 3, 4)
plt.xlabel(r"$\mathtt{RichDLLk}$", fontsize=12)
plt.ylabel(r"Candidates", fontsize=12)
plt.hist(dfs["RichDLLk"], bins=np.linspace(-150.0, 100.0, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)

plt.subplot(2, 3, 5)
plt.xlabel(r"$\mathtt{RichDLLp}$", fontsize=12)
plt.ylabel(r"Candidates", fontsize=12)
plt.hist(dfs["RichDLLp"], bins=np.linspace(-150.0, 100.0, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)

plt.subplot(2, 3, 6)
plt.xlabel(r"$\mathtt{RichDLLpk}$", fontsize=12)
plt.ylabel(r"Candidates", fontsize=12)
plt.hist(dfs["RichDLLp"] - dfs["RichDLLk"], bins=np.linspace(-100.0, 100.0, 101), histtype="step", lw=2, label=PARTICLE)
plt.legend(fontsize=10)
plt.yscale("log")

plt.show()

## Preprocessing

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import QuantileTransformer, StandardScaler

n_features = len(real_conditions)
n_flags = len(flag_conditions)
n_target = len(target)

prep_step_x = dict()

prep_step_x = ColumnTransformer(
    [
        ('features', QuantileTransformer(output_distribution='normal'), np.arange(n_features)),
        ('flags', "passthrough", n_features + np.arange(n_flags)),
    ]
).fit(dfs[conditions].values)

prep_step_y = dict()
# prep_step_y = StandardScaler().fit(dfs[target].values)
prep_step_y = QuantileTransformer(output_distribution='normal').fit(dfs[target].values)

In [ ]:
from preprocessing_utils import store_as_pickle

display(
    store_as_pickle(
        prep_step_x,
        f"PREPROCESSING_X",
        f"/tmp/lb-pidsim-train/models/Rich_{PARTICLE}_models/tX_{SAMPLE}.pkl",
    )
)
display(
    store_as_pickle(
        prep_step_y,
        f"PREPROCESSING_Y",
        f"/tmp/lb-pidsim-train/models/Rich_{PARTICLE}_models/tY_{SAMPLE}.pkl",
    )
)

## Train, test and validation

The dataset is split in:
 * *train* (50%) used for training the model
 * *test* (40%) used for measuring the performance of the model
 * *validation* (10%) used in combination with the train dataset to check for overtraining effects
 
Split data is stored on disk in chunks of 100 MB (before compression).

In [ ]:
DATA_PATH = os.environ.get("DATA_PATH", "/tmp/lb-pidsim-train/data")
os.makedirs(DATA_PATH, exist_ok=True)

In [ ]:
from preprocessing_utils import split_and_store, peek_from_dataset

from IPython.display import Markdown

display(Markdown(f"# {PARTICLE.capitalize()}"))

entries = split_and_store(
    sim[conditions + target],
    [
        (0.5, os.environ.get(f"TRAIN", f"{DATA_PATH}/Rich-{PARTICLE}-{SAMPLE}-train")),
        (0.4, os.environ.get(f"TEST", f"{DATA_PATH}/Rich-{PARTICLE}-{SAMPLE}-test")),
        (0.1, os.environ.get(f"VALIDATION", f"{DATA_PATH}/Rich-{PARTICLE}-{SAMPLE}-validation"))
    ],
    chunksize="80MB",
    features=conditions,
    labels=target,
    preprocessorX=prep_step_x,
    preprocessorY=prep_step_y,
)
    
display(pd.DataFrame(entries, index=["Train", "Test", "Validation"]))

X, y = peek_from_dataset(      
    f"TRAIN", 
    f"{DATA_PATH}/Rich-{PARTICLE}-{SAMPLE}-train",
)
 
## X Variables
plt.figure(figsize=(32, 5), dpi=100)

for i, x_var in enumerate(conditions):
    plt.subplot(1, 4, i+1)
    plt.xlabel(x_var, fontsize=12)
    plt.ylabel("Candidates", fontsize=12)
    plt.hist(X[:, i].numpy()[:100000], bins=100)

plt.show()

## RichDLL
plt.figure(figsize=(24, 10), dpi=100)

for i, y_var in enumerate(["RichDLLe", "RichDLLmu"]):
    plt.subplot(2, 3, i+1)
    plt.xlabel(y_var, fontsize=12)
    plt.ylabel("Candidates", fontsize=12)
    plt.hist(y[:, i].numpy()[:100000], bins=100)

plt.subplot(2, 3, 3)
plt.xlabel("RichDLLmue", fontsize=12)
plt.ylabel("Candidates", fontsize=12)
plt.hist(y[:, 1].numpy()[:100000] - y[:, 0].numpy()[:100000], bins=100)

for i, y_var in enumerate(["RichDLLk", "RichDLLp"], 2):
    plt.subplot(2, 3, i+2)
    plt.xlabel(y_var, fontsize=12)
    plt.ylabel("Candidates", fontsize=12)
    plt.hist(y[:, i].numpy()[:100000], bins=100)

plt.subplot(2, 3, 6)
plt.xlabel("RichDLLpk", fontsize=12)
plt.ylabel("Candidates", fontsize=12)
plt.hist(y[:, 3].numpy()[:100000] - y[:, 2].numpy()[:100000], bins=100)

plt.show()
